In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

colnames = [
            'isitu-LST',
            'Band1','Band2','Band3','Band4','Band6','Band7','Band8','Band9',
            'Band10' ,'Band11','Band12','Band13','Band14','Band15','Band16',
            '30daysBand3','30daysBand13',
            'SolarZA','SateZA','ESR','Height',
            'LandType',
            'insitu-HM','insitu-TD',
            'month']

def load_data(filename):  # 필요에 따라 레이블을 사용할수도 안할수도 있음

    df = pd.read_csv(filename)[colnames]

    X = df.loc[:, colnames[1:]].values
    y = df.loc[:, colnames[0]].values  # 레이블 값을 지난번과 다르게 0~8까지로 사용하도록 변경

    return X, y



X, y = load_data("과제2 결측치 제거-004.csv")

# X, y = load_data("/content/drive/MyDrive/wheather/data/remove_outlier.csv")
# X, y = load_data("/content/drive/MyDrive/wheather/data/과제2 결측제거/과제2 결측치 제거.csv")
# X, y = load_data("/content/drive/MyDrive/wheather/data/과제2 결측제거 5689/과제2 결측치 제거.csv")
# X, y = load_data("/content/drive/MyDrive/wheather/data/과제2 결측제거 extract_col/과제2 결측치 제거.csv")

print(X.shape, y.shape)

(10227754, 25) (10227754,)


In [ ]:
y.min(), y.max()
model = Sequential()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)
checkpoint = tf.keras.callbacks.ModelCheckpoint("lstm01.h5", monitor="val_loss", verbose=1 , save_best_only=True, mode="auto")

Adam = tf.keras.optimizers.Adam(clipnorm=1.)

model = Sequential()

# model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(64, int(len(colnames)-1))))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape(1), int(len(check_col)-1))))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(RepeatVector(64))

model.add(LSTM(256, input_shape=(int(len(colnames)-1), 1), return_sequences=True)) #input_shape은 x의 라벨값 6개 시퀀스 출력은 True 512차원 출력
model.add(Dropout(0.5)) #과적합 방지를 위한 드랍아웃 비율은 0.5
model.add(LSTM(256, return_sequences=True)) #LSTM 층  256차원출력+
model.add(Dropout(0.5)) #드랍아웃 층
model.add(LSTM(128)) #LSTM층 128차원 출력
model.add(Dense(100)) #활성화 함수
model.add(Dense(1, activation='relu')) #활성화 함수

# model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])
model.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mse'])
model.summary()

model.fit(X, y, epochs=2, batch_size=8192, verbose=1, callbacks=[early_stopping, checkpoint])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 25, 256)           264192    
                                                                 
 dropout_2 (Dropout)         (None, 25, 256)           0         
                                                                 
 lstm_4 (LSTM)               (None, 25, 256)           525312    
                                                                 
 dropout_3 (Dropout)         (None, 25, 256)           0         
                                                                 
 lstm_5 (LSTM)               (None, 128)               197120    
                                                                 
 dense_2 (Dense)             (None, 100)               12900     
                                                                 
 dense_3 (Dense)             (None, 1)                

2022-07-30 15:36:34.400679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 15:36:34.911271: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 15:36:34.998832: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 15:36:39.112543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 15:36:40.570335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 15:36:40.845812: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 15:36:40.958738: I tensorflow/core/grappler/optimizers/cust

1249/1249 [==============================] - 9272s 7s/step - loss: 13.1703 - mse: 13.1870
Epoch 2/2
1215/1249 [============================>.] - ETA: 4:37 - loss: 6.6824 - mse: 6.6932

In [ ]:
model.save("./pknu_lstm.h5")

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# X, y = load_data("/content/drive/MyDrive/wheather/data/only78.csv")
test_df = pd.read_csv("/content/drive/MyDrive/wheather/test/only78.csv")[colnames].astype(np.float32)
X = test_df.loc[:, colnames[1:]].values
y = test_df.loc[:, colnames[0]].values

y_score = model.predict(X)
print("MSE: " + str(mean_squared_error(y, y_score)))
print("RMSE: " + str(np.sqrt(mean_squared_error(y, y_score))))
print("MAE: " + str(mean_absolute_error(y, y_score)))
# y_pred = np.argmax(y_score, axis=1)
# pd.Series(y_pred, name=check_col[0]).to_csv("LSTM_result.csv")

In [ ]:
import pandas as pd


extract_col = ['YearMonthDayHourMinute', 'isitu-LST','insitu-TG','Band1','Band2','Band3','Band4','Band6','Band7','Band8','Band9','Band10','Band11','Band12','Band13','Band14','Band15','Band16','30daysBand3','30daysBand13','SolarZA','SateZA','ESR','Height','LandType','insitu-HM','insitu-TD']

df = pd.read_csv("/content/drive/MyDrive/wheather/data/과제2 결측치 제거_3.2g.csv")[extract_col]
df['month'] = pd.to_datetime(df['YearMonthDayHourMinute'], format='%Y%m%d%H%M%S').dt.strftime('%m').astype(int)
extract_col.append('month')

# df['LandType'] += 1

df.to_csv("/content/drive/MyDrive/wheather/data/과제2 결측치 제거_3.2g_2.csv")